In [ ]:
import pandas as pd
import networkx as nx
from community import community_louvain
from typing import Union
from tqdm.auto import tqdm
from collections import defaultdict
import os, json

# Jaccard similarity

In [ ]:
def jaccard_index(dict1: dict, dict2: dict, weight: Union[str, list[str]] = None, denomi_type: str = None) -> float:
    """
    Arguments:
     - set1, set2: edge/node set or graph with key and attributes
     - weight: edge/node feature to calculate Ruzicka similarity
     - denomi_type: denominator types. Current implementation only supports union type or 1 (without denominator)
    Return:
     - Jaccard similarity
    """
    s1 = set(dict1)
    s2 = set(dict2)
    intersect = s1 & s2
    if intersect == 0:
        return 0
    len_union = len(s1) + len(s2) - len(intersect)

    if denomi_type == 'union':
        denominator = len_union
    else:
        denominator = 1
    if weight is not None:
        intersect_len = weighted_intersection(dict1, dict2, intersect, weight)
        return intersect_len / denominator
    else:
        return len(intersect) / denominator
        
def weighted_intersection(dict1: dict, dict2: dict, intersect: set, weight: Union[str, list[str]]):
    """
    Argument:
     - dict1, dict2: dictionary with key and data -- key for intersection, data for similarity
     - intersect: list of the keys in intersection
    Return:
     - Cumulated Ruzicka similarity
    """
    total_len = 0.
    for elem in intersect:
        if isinstance(weight, list):
            total_len += get_vector_sum(dict1.get(elem), dict2.get(elem), weight)
        else:
            e1 = dict1.get(elem)[weight]
            e2 = dict2.get(elem)[weight]
            total_len += min(e1,e2) / max(e1,e2)
    return total_len

def get_vector_sum(e1, e2, weight_list):
    mins = 0.
    maxs = 0.
    for weight in weight_list:
        d1 = e1[weight]
        d2 = e2[weight]
        mins += min(d1, d2)
        maxs += max(d1, d2)
    return mins / maxs

## Function for year_lists.json

In [ ]:
def get_dict(graph: Union[nx.Graph, nx.DiGraph], main_character: str, level: int) -> dict:
    """
    Extract subgraphs ({1, 1.5}-ego networks) from graph
    """
    if main_character not in graph:
        return {}
    if level <= 1:
        _dict = {(u, v): d for u, v, d in graph.edges(main_character, data = True)}
        if isinstance(graph, nx.DiGraph):
            _dict.update({(u, v): d for u, v, d in graph.in_edges(main_character, data = True)})
    else:
        neighbors = list(graph.neighbors(main_character)) + [main_character]
        subgraph = graph.subgraph(neighbors)
        _dict = {(u,v): d for u,v,d in subgraph.edges(data = True)}
    return _dict
        

def get_comparison(graph1: Union[nx.Graph, nx.DiGraph], graph2: Union[nx.Graph, nx.DiGraph],
                   denomi_type: str, level = 1, weight: Union[str, list[str]] = 'weight') -> dict:
    """
    Calculate distances of each characters in two graphs
    """
    nodes = set(graph1.nodes) | set(graph2.nodes)
    _dict = {}
    for node in nodes:
        g1 = get_dict(graph1, node, level)
        g2 = get_dict(graph2, node, level)
        len_union = len(set(g1) | set(g2))
        sim = jaccard_index(g1, g2, weight, denomi_type)
        dist = 1 - sim if denomi_type == 'union' else len_union - sim
        _dict[node] = dist
    return _dict

def convert_yearlist_to_dissim(_dict: dict) -> dict:
    """
    Convert year_lists.json format into the dissim.json format
    """
    dissims = defaultdict(list)
    _max = -1e9
    _min = 1e9
    for item in _dict:
        dists = item['distance']
        _from = item['from']
        _to = item['to']
        for node, dist in dists.items():
            dissims[node].append({
                "distance": dist,
                "from": _from,
                "to": _to
            })
            _max = max(_max, dist)
            _min = min(_min, dist)
    dissims["metadata"].append({
        "max": _max,
        "min": _min
    })
    return dict(dissims)

## VisPub data Processing

Download VisPub data from https://vispubdata.org/

In [ ]:
data = pd.read_csv("./data/VisPub/vispubdata.csv") #Load vispub data
yearly_graphs = {}
# Iterate through the data and create graphs
for index, row in data.iterrows():
    year = row["Year"]
    if year not in yearly_graphs:
        yearly_graphs[year] = nx.Graph()
    authors = row["AuthorNames-Deduped"].split(';')
    graph = yearly_graphs[year]
    for i in range(len(authors)):
        for j in range(i + 1, len(authors)):
            if not graph.has_edge(authors[i], authors[j]):
                graph.add_edge(authors[i], authors[j], weight=0)
            graph[authors[i]][authors[j]]['weight'] += 1
sorted_yearly_graphs = {year: yearly_graphs[year] for year in sorted(yearly_graphs)}

In [ ]:
"""
    Generate ego/year_lists.json and ego/dissim.json --> set level 1
"""
lbd = min(sorted_yearly_graphs.keys())
ubd = max(sorted_yearly_graphs.keys())

is_ego = True

level = 2 - is_ego
save_results = True

year_lists = []
for t1 in tqdm(range(lbd, ubd)):
    for t2 in range(t1 + 1, ubd + 1):
        _dict = get_comparison(sorted_yearly_graphs[t1], sorted_yearly_graphs[t2], denomi_type = None, level = level, weight = 'weight')
        year_lists.append({
            'distance': _dict,
            'from': t1,
            'to': t2
        })

if save_results:
    if not os.path.exists('ego'):
        os.makedirs('ego')
    json.dump(year_lists, open('ego/year_lists.json', 'w'))
    json.dump(convert_yearlist_to_dissim(year_lists), open('ego/year_lists.json', 'w'))

In [ ]:
"""
    Generate neighbor/year_lists.json and neighbor/dissim.json --> set level > 1
"""
lbd = min(sorted_yearly_graphs.keys())
ubd = max(sorted_yearly_graphs.keys())

is_ego = False

level = 2 - is_ego
save_results = True

year_lists = []
for t1 in tqdm(range(lbd, ubd)):
    for t2 in range(t1 + 1, ubd + 1):
        _dict = get_comparison(sorted_yearly_graphs[t1], sorted_yearly_graphs[t2], denomi_type = None, level = level, weight = 'weight')
        year_lists.append({
            'distance': _dict,
            'from': t1,
            'to': t2
        })

if save_results:
    if not os.path.exists('neighbor'):
        os.makedirs('neighbor')
    json.dump(year_lists, open('neighbor/year_lists.json', 'w'))
    json.dump(convert_yearlist_to_dissim(year_lists), open('neighbor/year_lists.json', 'w'))

## Community Generation

In [ ]:
"""
Generate comm_lists.json and partition_lists.json
"""
def get_partition_subgraph(graph, partition_dict, target_node):
    comm = partition_dict[target_node]
    subgraph_nodes = [k for k, v in partition_dict.items() if v == comm]
    return graph.subgraph(subgraph_nodes)

save_results = True

partition_list = {}
comm_list = {}

for t, graph in sorted_yearly_graphs.items(): 
    G = graph.to_undirected()
    partition = community_louvain.best_partition(G)
    comm = defaultdict(list) # dictionary which has list-type values for detailed implementation, please check defaultdict
    for k, v in partition.items():
        comm[v].append(k)
    partition_list[t] = {k: str(t) + "-" + str(v) for k,v in partition.items()}
    comm_list[t] = {str(t) + "-" + str(k): v for k, v in dict(comm).items()} # change defaultdict to python original dictionary type
if save_results:
    json.dump(partition_list, open('partition_lists.json', 'w'))
    json.dump(comm_list, open('comm_lists.json', 'w'))